## Hometask 1 - связь Python с программами на C/C++

1. Реализовать на языке C/C++ классические операции перемножения квадратных матриц и умножения матрицы на вектор (15%)

Решение: см. код в текушей директории:
- файл `matrix_operations` - с операциями над матрицами
- файлы `main_matrix_multiply`, `main_matrix_vector_multiply` - с `main` функциями для экспериментов.

2. Разделить программу на несколько модулей и провести сборку через статическую линковку (25%)

In [2]:
! g++ -c -g matrix_operations.c
! g++ -c -g main_matrix_multiply.c
! g++ matrix_operations.o main_matrix_multiply.o -o main_matrix_multiply_program_g

! g++ -c -O3 matrix_operations.c
! g++ -c -O3 main_matrix_multiply.c
! g++ matrix_operations.o main_matrix_multiply.o -o main_matrix_multiply_program_O3

clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]
clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]
clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]
clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]


In [11]:
! g++ -c -g matrix_operations.c
! g++ -c -g main_matrix_vector_multiply.c
! g++ matrix_operations.o main_matrix_vector_multiply.o -o main_matrix_vector_multiply_program_g

! g++ -c -O3 matrix_operations.c
! g++ -c -O3 main_matrix_vector_multiply.c
! g++ matrix_operations.o main_matrix_vector_multiply.o -o main_matrix_vector_multiply_program_O3

clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]
clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]
clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]
clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]


3. Подготовьте две сборки с флагами -g и -O3 и измерьте времена выполнения операций с N = 512, 1024, . . ., 4096 (20%).

При данных $N$ машина считает довольно медленно. Уменьшил величины проверяемых значений.

In [13]:
import numpy as np
import os
import pandas as pd
import subprocess
import timeit

from pathlib import Path


root = Path(os.getcwd())
def run_program(program_name, N):
    subprocess.run([str(root / program_name), str(N)])


programs = {
    'Умножение матриц (-g)': 'main_matrix_multiply_program_g',
    'Умножение матриц (-O3)': 'main_matrix_multiply_program_O3',
    'Умножение матрицы на вектор (-g)': 'main_matrix_vector_multiply_program_g',
    'Умножение матрицы на вектор (-O3)': 'main_matrix_vector_multiply_program_O3',
}

res_c = pd.DataFrame(index=programs.keys())
n_elements = [2**i for i in range(6, 12)]
for n_element in n_elements:
    res_c[n_element] = np.nan
    for experiment_name, program_name in programs.items():
        time = timeit.timeit(lambda: run_program(program_name, n_element), number=4)
        res_c.loc[experiment_name, n_element] = time
        
res_c

,64,128,256,512,1024,2048
Умножение матриц (-g),0.030999,0.056164,0.277642,1.867352,17.715019,268.069243
Умножение матриц (-O3),0.027012,0.030242,0.090274,0.613639,5.561854,106.334143
Умножение матрицы на вектор (-g),0.027510,0.019615,0.021149,0.025896,0.040983,0.095335
Умножение матрицы на вектор (-O3),0.025274,0.018695,0.018952,0.020594,0.027104,0.057569


4. Выполните вызов процедуры из Python через Ctypes/Cython/PyBind11 и измерьте времена (40%)

Получилось реализовать только Ctypes:

In [14]:
!g++ -O3 -shared matrix_operations.o -o matrix_operations.so

In [15]:
import ctypes
import numpy as np
import time


def run_ctypes_matrix_multiply(n):
    # Load the shared library
    lib = ctypes.CDLL('./matrix_operations.so')
    
    # Define the function signatures
    matrix_multiply = lib.matrix_multiply
    matrix_multiply.argtypes = [ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double), ctypes.c_int, ctypes.POINTER(ctypes.c_double)]
    matrix_multiply.restype = None
    
    A = np.empty((n, n), dtype=np.double)
    B = np.empty((n, n), dtype=np.double)
    
    # Measure execution time of matrix multiplication
    C = np.empty((n, n), dtype=np.double)
    matrix_multiply(A.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                    B.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                    ctypes.c_int(n),
                    C.ctypes.data_as(ctypes.POINTER(ctypes.c_double)))
    
    return C
    

def run_ctypes_matrix_vector_multiply(n):
    # Load the shared library
    lib = ctypes.CDLL('./matrix_operations.so')
    
    # Define the function signatures
    matrix_vector_multiply = lib.matrix_vector_multiply
    matrix_vector_multiply.argtypes = [ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double), ctypes.c_int, ctypes.POINTER(ctypes.c_double)]
    matrix_vector_multiply.restype = None
    
    A = np.ones((n, n), dtype=np.double)
    x = np.full(n, 2.0, dtype=np.double)
    y = np.empty(n, dtype=np.double)
    
    # Measure execution time of matrix multiplication
    matrix_vector_multiply(A.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                      x.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                      ctypes.c_int(n),
                      y.ctypes.data_as(ctypes.POINTER(ctypes.c_double)))
    
    return y
    

programs = {
    'Умножение матриц (Ctypes)': run_ctypes_matrix_multiply,
    'Умножение матрицы на вектор (Ctypes)': run_ctypes_matrix_vector_multiply,
}

res_ctypes = pd.DataFrame(index=programs.keys())
for experiment_name, func in programs.items():
    n_repeat = 5
    for n_element in n_elements:
        sum_time = 0
        for i_repeat in range(n_repeat):
            start = time.time()
            func(n_element)
            end = time.time()
            sum_time += end - start
        res_ctypes.loc[experiment_name, n_element] = sum_time / n_repeat

Проверим корректность вычисления:

In [20]:
lib = ctypes.CDLL('./matrix_operations.so')
    
matrix_multiply = lib.matrix_multiply
matrix_multiply.argtypes = [ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double), ctypes.c_int, ctypes.POINTER(ctypes.c_double)]
matrix_multiply.restype = None

A = np.array([[1, 2], [3, 4]], dtype=np.double)
B = np.array([[1, 1], [2, 2]], dtype=np.double)

C = np.empty((2, 2), dtype=np.double)
matrix_multiply(A.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                B.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                ctypes.c_int(2),
                C.ctypes.data_as(ctypes.POINTER(ctypes.c_double)))

C

array([[ 5.,  5.],
       [11., 11.]])

# Результаты в сравнении

In [21]:
pd.concat([res_c, res_ctypes])

,64,128,256,512,1024,2048
Умножение матриц (-g),0.030999,0.056164,0.277642,1.867352,17.715019,268.069243
Умножение матриц (-O3),0.027012,0.030242,0.090274,0.613639,5.561854,106.334143
Умножение матрицы на вектор (-g),0.027510,0.019615,0.021149,0.025896,0.040983,0.095335
Умножение матрицы на вектор (-O3),0.025274,0.018695,0.018952,0.020594,0.027104,0.057569
Умножение матриц (Ctypes),0.007295,0.001872,0.016440,0.154716,1.465752,25.927263
Умножение матрицы на вектор (Ctypes),0.000091,0.000110,0.000100,0.000413,0.001918,0.008194
